# Load dataset

In [3]:
from transformers import AutoTokenizer
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from sentence_transformers import SentenceTransformer

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
vietnamese_history_dataset_path = "../../datasets/vietnamese_history_dataset/data.json"
df = pd.read_json(vietnamese_history_dataset_path)
df.head()

,title,content,type
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title


# Load tokenizer model

In [4]:
model = BertForSequenceClassification.from_pretrained(
    "trituenhantaoio/bert-base-vietnamese-uncased",
    cache_dir="../../tokenizer_models/"
)
tokenizer = BertTokenizer.from_pretrained(
    "trituenhantaoio/bert-base-vietnamese-uncased",
    cache_dir="../../tokenizer_models/"
)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at trituenhantaoio/bert-base-vietnamese-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/207k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# test thử trên một mẫu
text = df.iloc[0]["content"]
print(text)

inputs = tokenizer(
    text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

In [ ]:
print()